In [32]:
from ase import Atoms
from ase.io import read
import xcquinox as xce
import torch, jax, optax
import numpy as np
import equinox as eqx
import jax.numpy as jnp
import pyscfad as psa
import os, sys
from pyscf import dft, scf, gto, df
from pyscf.pbc import scf as scfp
from pyscf.pbc import gto as gtop
from pyscf.pbc import dft as dftp
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='false'
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR'] = 'platform'

from mp_api.client import MPRester

from ase.build import bulk

In [35]:
a1 = bulk('Si', a=3.867114, b=3.867114, c=3.867114, alpha=60)

In [36]:
a1.cell

Cell([[0.0, 1.933557, 1.933557], [1.933557, 0.0, 1.933557], [1.933557, 1.933557, 0.0]])

In [49]:
mpr = MPRester(api_key = None)
mpid = 'mp-149'
# ret = mpr.get_bandstructure_by_material_id(mpid)
# rets = ret.structure.as_dict()
ret = mpr.get_structure_by_material_id(mpid, conventional_unit_cell=False)
rets = ret.as_dict()
at_coor_xyz = [ (i['species'][0]['element'], [-j for j in i['xyz']]) for i in rets['sites']]
at_coor_abc = [ (i['species'][0]['element'], [rets['lattice']['a']*j for j in i['abc']]) for i in rets['sites']]
cella = -np.asarray(rets['lattice']['matrix'])

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [45]:
at_coor_xyz, cella

([('Si', [4.1016945, 4.1016945, 4.1016945]), ('Si', [-0.0, -0.0, -0.0])],
 array([[ 2.734463,  2.734463, -0.      ],
        [ 2.734463, -0.      ,  2.734463],
        [-0.      ,  2.734463,  2.734463]]))

In [46]:
cell = gtop.Cell()
cell.atom = at_coor_xyz
cell.a = cella.T
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.exp_to_discard = 0.1
cell.build()
kpts = cell.make_kpts([2,2,2])
mf = scfp.KRHF(cell, kpts=kpts)
e = mf.kernel()

WARNING!
  Lattice are not in right-handed coordinate system. This can cause wrong value for some integrals.
  It's recommended to resort the lattice vectors to
a = [[ 5.16738617  5.16738617 -0.        ]
 [-0.          5.16738617  5.16738617]
 [ 5.16738617 -0.          5.16738617]]



KeyboardInterrupt: 

In [6]:
cella

array([[ 1.786855,  1.786855, -0.      ],
       [ 1.786855, -0.      ,  1.786855],
       [-0.      ,  1.786855,  1.786855]])